# 🦠 Interactive SEIR Model Simulation

## Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## SEIR Model Parameters: Configure Simulation Settings

In [ ]:
N = 10_000_000   # Total population
E0 = 10_000      # Initial number of exposed individuals
I0 = 2_000       # Initial number of infectious individuals
R0_value = 2.0   # Basic reproduction number (R0)
alpha = 1 / 5.5  # Rate at which exposed individuals become infectious
gamma = 1 / 3.0  # Recovery rate
days = 140       # Number of days to simulate
step = 0.1       # Time step for numerical integration

## Explicit Euler method for numerical integration

In [ ]:
def euler(f, t0, x0, t1, h):
    """
    Solves an ODE using the explicit Euler method.

    Parameters:
        f  : function - The derivative function of the system.
        t0 : float - Initial time.
        x0 : array-like - Initial state vector.
        t1 : float - Final time.
        h  : float - Step size.

    Returns:
        List of time steps and corresponding state values.
    """
    t = t0
    x = x0
    trajectory = [[t, x]]

    for k in range(0, 1 + int((t1 - t0) / h)):
        t = t0 + k * h
        x = x + h * f(t, x)
        trajectory.append([t, x])

    return trajectory

## SEIR Model equations

In [ ]:
def SEIR_model(beta, alpha, gamma):
    """
    Defines the SEIR model differential equations.

    Parameters:
        beta  : float - Transmission rate.
        alpha : float - Rate at which exposed individuals become infectious.
        gamma : float - Recovery rate.

    Returns:
        Function representing the SEIR model.
    """
    def f(t, x):
        S, E, I, R = x
        return np.array([
            -beta * S * I,              # dS/dt
            beta * S * I - alpha * E,   # dE/dt
            alpha * E - gamma * I,      # dI/dt
            gamma * I                   # dR/dt
        ])
    return f

## SEIR Simulation function

In [ ]:
def SEIR_simulation(beta, alpha, gamma, E0, I0, days, step=0.1):
    """
    Runs an SEIR model simulation.

    Parameters:
        beta  : float - Transmission rate.
        alpha : float - Rate at which exposed individuals become infectious.
        gamma : float - Recovery rate.
        E0    : float - Initial proportion of exposed individuals.
        I0    : float - Initial proportion of infectious individuals.
        days  : int - Number of days to simulate.
        step  : float - Time step for numerical integration.

    Returns:
        Time array and arrays of S, E, I, R values.
    """
    S0 = 1.0 - E0 - I0  # Initial proportion of susceptible individuals
    x0 = np.array([S0, E0, I0, 0.0])  # Initial state
    f = SEIR_model(beta, alpha, gamma)
    result = euler(f, 0, x0, days, step)
    t, x = zip(*result)
    return np.array(t), np.array(x).T

## Plotting the SEIR Simulation results

In [ ]:
def plot_SEIR():
    beta = R0_value * gamma
    E0_frac = E0 / N
    I0_frac = I0 / N
    t, (S, E, I, R) = SEIR_simulation(beta, alpha, gamma, E0_frac, I0_frac, days, step)

    plt.style.use('seaborn-v0_8-poster')
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(t, S, label='Susceptible (S)', color='blue')
    ax.plot(t, E, label='Exposed (E)', color='#FFA500', linestyle='--')
    ax.plot(t, I, label='Infectious (I)', color='red')
    ax.plot(t, R, label='Recovered (R)', color='green', linestyle='--')

    ax.set_title("SEIR Model Simulation", fontsize=16)
    ax.set_xlabel("Days", fontsize=12)
    ax.set_ylabel("Proportion of Population", fontsize=12)
    ax.legend(fontsize=12, fancybox=True, shadow=True)
    ax.grid(linestyle=':', linewidth=2.0, color='#808080')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_SEIR()